In [17]:
#code is from https://youtu.be/2kSPbH4jWME by Dataquest, seen feb. 24. 2023 
#here we import liberies and elements form those functions
import ipywidgets as widgets
from IPython.display import display
from threading import Thread
from queue import Queue

#the queues are threads so the run in the background
#a queue for messages
messages = Queue()

#a queue for recordings
recordings = Queue()

#Here we create a button called "record_button" and determin it as a widgets so we can interacet with it.
#then we set the properties for, such as style, color, when its off and its icon.
record_button = widgets.Button(
    description="Record",
    disabled = False,
    button_style="success",
    icon="microphone"
)

#here we do the same thing, as the one above, but for a stop button insted
stop_button = widgets.Button(
    description="Stop",
    disabled = False,
    button_style="warning",
    icon="stop"
)

#this is the function we run to get a output
output = widgets.Output()

#this is the functions where the program understan what happens when start recording and stop recording 
def start_recording(data):
    
   #this will add a message to teh queue
    messages.put(True)
    
    with output:
        display("starting...")
        
        #here is where we telle the threads that it should reecord the microphone
        record = Thread(target=record_microphone)
        record.start()
        
        #here we telle the threads to transcribe the audio 
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()
    
def stop_recording(data):
    with output:
        messages.get()
        display("Stopped.")
        
#here we tell it what happens when the buttons are pressed
record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

#this displays the buttons and the trancript
display(record_button,stop_button,output)


Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output()

Exception in thread Thread-11 (speech_recognition):
Traceback (most recent call last):
  File "c:\Users\Nicol\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\Nicol\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Nicol\AppData\Local\Temp\ipykernel_9920\4087974563.py", line 17, in speech_recognition
  File "c:\Users\Nicol\AppData\Local\Programs\Python\Python311\Lib\subprocess.py", line 466, in check_output
    return run(*popenargs, stdout=PIPE, timeout=timeout, check=True,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nicol\AppData\Local\Programs\Python\Python311\Lib\subprocess.py", line 571, in run
    raise CalledProcessError(retcode, process.args,
subprocess.CalledProcessError: Command 'python recasepunc/recasepunc.py predict recasepunc/checkpoint' returned non-zero exit status 2.


In [8]:
import pyaudio

p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

p.terminate()


{'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'Mikrofon KONIG (2- USB Audio De', 'hostApi': 0, 'maxInputChannels': 1, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'Mikrofon (Realtek(R) Audio)', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'Headset Microphone (Oculus Virt', 'hostApi': 0, 'maxInputChannels'

In [9]:
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 20
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()
    
    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    rate= FRAME_RATE,
                    input = True,
                    input_device_index=1,
                    frames_per_buffer=chunk)
    frames = []
    
    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            Frames = []
        
    stream.stop_stream()
    stream.close()
    p.terminate()

In [16]:
import subprocess
import json
from vosk import Model, KaldiRecognizer

Model = Model(model_name="vosk-model-en-us-0.22")
rec = KaldiRecognizer(Model,FRAME_RATE)
rec.SetWords(True)

def speech_recognition(output):
    while not messages.empty():
        frames = recordings.get()
        
        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text = json.loads(result)["text"]
        
        cased = subprocess.check_output("python recasepunc/recasepunc.py predict recasepunc/checkpoint", shell=True,text=True,input=text)
        output.append_stdout(cased)